## Single-cell RNA data on Minimal Residual disease for melanoma!
![MRD](mrd.png "Minimal residual Disease for melanoma")

## Learning Outcome

After this session you will be able to load and pre-process your multi-omics data to generate lower-dimensional embeddings. 

### Quality Control
![qc](mrd1.png "Quality Control for MRD")

### Generate lower-dimensional embeddings

We will perform dimensionality reduction and generate lower-dimensional embeddings of the single-cell RNAseq data using two methods:
* PCA (https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html)
* Neural Networks (https://pytorch.org/tutorials/beginner/basics/buildmodel_tutorial.html)


In [2]:
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns 
sns.set_style('dark')

# ====== Scikit-learn imports ======

from sklearn.svm import SVC
from sklearn.metrics import (
    auc,
    roc_curve,
    ConfusionMatrixDisplay,
    f1_score,
    balanced_accuracy_score,
)
from sklearn.preprocessing import StandardScaler, LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA

## ====== Torch imports ======
import torch
import torch.nn as nn 
import torch.nn.functional as F
import torch.optim as optim
from lightning.pytorch.utilities.types import OptimizerLRScheduler
import torch.utils.data
from torch.utils.data import TensorDataset, DataLoader

import lightning, lightning.pytorch.loggers

### Read data

In [3]:
df = pd.read_csv("../data/GSE116237_forQ.csv")
print(df.shape)
labels = pd.read_csv("../data/GSE116237 filtered labels.csv")
labels.drop('Unnamed: 0', axis=1, inplace=True)
labels.columns = ['Cells', 'Labels']
labels_filtered = labels[labels['Cells'].isin(list(df['Cells']))]
labels_filtered['Labels'] = [x.split(' ')[1] for x in labels_filtered['Labels']]
df = pd.merge(df, labels_filtered, on='Cells', how='inner')
df['Labels'] = df['Labels'].map({'T0': 0, 'phase2': 1})
y = np.array(df['Labels'])
X = df[df.columns[1:-1]].values

num_samples = X.shape[0]
num_feats = X.shape[1]

(369, 2002)


/tmp/ipykernel_3025929/715312502.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  labels_filtered['Labels'] = [x.split(' ')[1] for x in labels_filtered['Labels']]


### Split into training and testing

In [4]:
X_working, X_held_out, y_working, y_held_out = train_test_split(X,
                                                    y,
                                                    train_size=0.8,
                                                    shuffle=True)

### PCA

In [5]:
output_dim = 5
pca = PCA(n_components=output_dim)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_working)
embedding_train = pca.fit_transform(X_train_scaled)
X_test_scaled = scaler.fit_transform(X_held_out)
embedding_test = pca.fit_transform(X_test_scaled)
print("Training data dimensions: ", X_working.shape)
print("Embedded train dimensions: ", embedding_train.shape)
print("Testing data dimensions: ", X_held_out.shape)
print("Embedded test dimensions: ", embedding_test.shape)

Training data dimensions:  (295, 2001)
Embedded train dimensions:  (295, 5)
Testing data dimensions:  (74, 2001)
Embedded test dimensions:  (74, 5)


### Neural Networks

Define the network in Pytorch lightning

In [6]:
embedder = torch.nn.Sequential(
            nn.Linear(num_feats,256),
            nn.LeakyReLU(),
            nn.Linear(256,64),
            nn.LeakyReLU(),
            nn.Linear(64,32), 
            nn.LeakyReLU(),
            nn.Linear(32,16), 
            nn.LeakyReLU(),
            nn.Linear(16,output_dim),
            nn.LeakyReLU(), 
            ) 

classifier = torch.nn.Sequential(
            nn.Linear(output_dim,1),
            nn.Softmax(dim=1)
            )
class BinaryClassifierModel(lightning.LightningModule):
    def __init__(self, embedder, classifier, input_dim,learning_rate=1e-3):
        super().__init__()
        self.input_dim = input_dim
        self.embedder = embedder
        self.classifier = classifier
        self.learning_rate = learning_rate
        self.loss_fun = nn.BCELoss()
        
    def train_dataloader(self):
        return DataLoader(self.train_data, batch_size=32, shuffle=True)

    def val_dataloader(self):
        return DataLoader(self.val_data, batch_size=32)  # No shuffling for validation
    
    def forward(self, X): 
        x = self.embedder(X)
        x = self.classifier(x) 
        return x 
    
    def training_step(self, batch, batch_idx):
        x, y = batch
        y = y.unsqueeze(1)
        y_float = y.float()
        x_embedder = self.embedder(x)
        y_hat = self.classifier(x_embedder)
        #y_hat = torch.argmax(y_hat, dim=1)
        loss = self.loss_fun(y_hat, y_float)
        self.log("train_loss", loss, 
                prog_bar=True, 
                logger=True)
        return loss
    
    def validation_step(self, batch, batch_idx):
        x, y = batch
        y = y.unsqueeze(1)
        y_float = y.float()
        x_embedder = self.embedder(x)
        y_hat = self.classifier(x_embedder)
        val_loss = self.loss_fun(y_hat, y_float)
        f1score = f1_score(y_hat, y)
        #print(f1score)
        #print(val_loss)
        self.log("val_loss", val_loss, prog_bar=False, logger=True)  # Log on epoch end
        return val_loss

        
    def configure_optimizers(self):
        return torch.optim.Adam(self.classifier.parameters(), lr=self.learning_rate)
    
def prepare_data(X_train, y_train, X_val, y_val):
    # Assuming X and y are NumPy arrays

    train_data = TensorDataset(torch.tensor(X_train, dtype=torch.float32), 
                        torch.tensor(y_train, dtype=torch.float32))
    val_data = TensorDataset(torch.tensor(X_val, dtype=torch.float32), 
                        torch.tensor(y_val, dtype=torch.float32))
    
    return train_data, val_data 

Compute the embeddings using the network and get lower dimension embeddings in the form of matrices for downstream QML tasks

In [7]:
f1s = []
embeddings_train = []
embeddings_test = []
train_labels = []
test_labels = [] 

num_iter = 1
for i in range(num_iter): 

    X_train, X_test, y_train, y_test = train_test_split(X_working,
                                                        y_working,
                                                        train_size=0.9,
                                                        shuffle=True)

    num_epochs = 40
    model = BinaryClassifierModel(embedder, classifier, input_dim=num_feats)
    model.train_data, model.val_data = prepare_data(X_train, y_train, X_test, y_test)  # Prepare data for training
    logger = lightning.pytorch.loggers.TensorBoardLogger(save_dir=".",name="original_classifier")
    # Train the model
    trainer = lightning.Trainer(max_epochs=num_epochs, 
                                logger=logger)  # Adjust progress bar refresh rate as needed
    trainer.fit(model)
    model.eval()
    embedded_test = model.embedder(torch.tensor(X_test, dtype=torch.float32))
    y_pred = model.classifier(embedded_test)
    y_pred_proba = y_pred.detach().cpu().numpy()
    y_pred_class = np.round(y_pred_proba)

    f1 = f1_score(y_test, y_pred_class)
    f1s.append(f1)
    
    embedded_train = model.embedder(torch.tensor(X_train, dtype=torch.float32)).detach().numpy()
    embeddings_train.append(embedded_train)
    embeddings_test.append(embedded_test.detach().numpy())
    train_labels.append(y_train)
    test_labels.append(y_test)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
2024-07-12 13:12:26.638257: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-07-12 13:12:26.651571: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:479] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-12 13:12:26.671424: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:10575] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-12 13:12:26.671449: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1442] Unable to register cuBLAS factory: Attempting to register factory for plugin cuB

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/home/abose/QMLOmics/SessionIII_MultiOmics/.venv/lib/python3.10/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:424: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=191` in the `DataLoader` to improve performance.
/home/abose/QMLOmics/SessionIII_MultiOmics/.venv/lib/python3.10/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:424: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=191` in the `DataLoader` to improve performance.
/home/abose/QMLOmics/SessionIII_MultiOmics/.venv/lib/python3.10/site-packages/lightning/pytorch/loops/fit_loop.py:298: The number of training batches (9) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.


Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=40` reached.


### Save the embeddings

In [11]:
fname = "MRD"

for i,x in enumerate(embeddings_train): 
    fname_train = fname + "_iter" + str(i) + "_train_embedding"
    np.save(f"checkpoints/{fname}/{fname_train}", x)

for i,x in enumerate(train_labels): 
    fname_train_y = fname + "_iter" + str(i) + "_train_labels"
    np.save(f"checkpoints/{fname}/{fname_train_y}", x)

for i,x in enumerate(embeddings_test): 
    fname_test = fname + "_iter" + str(i) + "_test_embedding"
    np.save(f"checkpoints/{fname}/{fname_test}", x)
    
for i,x in enumerate(test_labels): 
    fname_test_y = fname + "_iter" + str(i) + "_test_labels"
    np.save(f"checkpoints/{fname}/{fname_test_y}", x)
